## Gait Video Study 
### From the camera calibration intrinsic and extrinsic matrix, computing the 3D from 2D coordinates for both front camera and side camera looks 

In [1]:
import numpy as np
import cv2
import os
import glob
import matplotlib.pyplot as plt
import pandas as pd
import time

In [2]:
path = 'C:\\Users\\purpl\\Box\\Gait Video Project\\CameraCalibration\\'
raw_path = 'C:\\Users\\purpl\\Box\\Gait Video Project\\GaitVideoData\\video\\openpose_data' 
frame_path3d = 'C:\\Users\\purpl\\Box\\Gait Video Project\\GaitVideoData\\video\\3D_data'

In [3]:
#Configuration for which to run the code for 
cohorts = ['\\HOA', '\\MS', '\\PD', '\\ExtraHOA']
trials = ['\\beam_walking', '\\walking']
cameras = ['\\feet\\', '\\lower_body\\']

In [4]:
## Run only once to create directories 
#To create all directories for saving the 3D
# for cohort in cohorts:
#     for trial in trials[1:]:
#         for camera in cameras[1:]:    
#             frame_path2d = frame_path+cohort+trial+camera #Path to save the frames to 
#             videos = os.listdir(frame_path2d)
#             for video in videos:
#                 if not os.path.exists(frame_path3d+cohort+trial+camera+video):
#                     os.makedirs(frame_path3d+cohort+trial+camera+video)

### Front look camera (_1)

In [5]:
#Use the post processed 2D files from hereon
# Save the new file in 3D_data folder. Make sure to preserve the confidence score of the estimated coordinate
#since we will do weighted combining based on the confidence score later on estimated 3D coordinates
# Load previously saved intrinsic and extrinsic matrices 

#Retrieving the lower body intrinsic and extrinsic coordinates 
frontlook_mtx = np.load(path+'June\\frontlook\\calibrate\\frontlook_camera_matrix.npy')
frontlook_dist = np.load(path+'June\\frontlook\\calibrate\\frontlook_distortion_matrix.npy')
frontlook_rvecs = np.load(path+'June\\frontlook\\calibrate\\frontlook_rotation_matrix.npy')
frontlook_tvecs = np.load(path+'June\\frontlook\\calibrate\\frontlook_translation_matrix.npy')

print ('Computed camera matrix\n', frontlook_mtx)
print ('Computed distortion matrix\n', frontlook_dist)
print ('Computed rotation vector\n', frontlook_rvecs)
print ('Computed translation vector\n', frontlook_tvecs)

rotM = cv2.Rodrigues(frontlook_rvecs)[0]

Computed camera matrix
 [[560.02665252   0.         402.60078829]
 [  0.         869.92913382 252.54590069]
 [  0.           0.           1.        ]]
Computed distortion matrix
 [[-4.70342049e-01  9.05259576e+00  4.59566683e-03  2.16735826e-02
  -5.32980337e+01]]
Computed rotation vector
 [[ 0.04456356]
 [ 3.01837349]
 [-0.90960737]]
Computed translation vector
 [[  31.90938981]
 [-232.49838609]
 [ 249.25996024]]


In [6]:
#Dimensions of the image 
w = 800
h = 448

#Order of saved coordinates 
order = ['right hip', 'right knee', 'right ankle', 'left hip', 'left knee', 'left ankle', 'left toe 1', 'left toe 2', \
         'left heel', 'right toe 1', 'right toe 2', 'right heel']

In [7]:
def convert2d_to_3d(coord, scale, shift_array):
    frontlook_tvecs1 = frontlook_tvecs-np.array(shift_array)
    col = coord[0]
    row = coord[1]
    u=col-w/2
    v=h/2-row
    p = np.matmul(np.linalg.pinv(frontlook_mtx), np.array([col, row, 1]))
    return (np.matmul(np.matrix(rotM).T, (scale*p.reshape(-1,1) - np.matrix(frontlook_tvecs1))).T)

In [37]:
frame_path = raw_path+'\\ExtraHOA'+'\\walking\\'+'lower_body\\' #Path to save the frames to 
os.listdir(frame_path)

['InkedGVS_102_W_T1_1_Trim',
 'InkedGVS_112_W_T1_1_Trim',
 'InkedGVS_113_W_T1_1_Trim',
 'InkedGVS_115_W_T1_1_Trim',
 'InkedGVS_123_W_T1_1_Trim',
 'InkedGVS_124_W_T1_1_Trim']

In [48]:
#Make a dictionary of arrays for all videos, where the key is the video name and value is the correponding array
array1 = np.array([[5], [5], [-520]])
array2 = np.array([[5], [-60], [-530]])
array3 = np.array([[5], [-80], [-530]])
array4 = np.array([[5], [-120], [-530]])

arrays = {
  **dict.fromkeys(['InkedGVS_212_T_T1_1_Trim',  'InkedGVS_212_T_T2_1_Trim', 'InkedGVS_213_T_T1_1_Trim',\
 'InkedGVS_213_T_T2_1_Trim', 'InkedGVS_214_T_T1_1_Trim', 'InkedGVS_214_T_T2_1_Trim',\
    'InkedGVS_215_T_T1_1_Trim', 'InkedGVS_215_T_T2_1_Trim', 'InkedGVS_216_T_T1_1_Trim', 'InkedGVS_216_T_T2_1_Trim', \
  'InkedGVS_218_T_T1_1_Trim', 'InkedGVS_218_T_T2_1_Trim', 'InkedGVS_219_T_T1_1_Trim', 'InkedGVS_219_T_T2_1_Trim', \
  'InkedGVS_212_W_T1_1_Trim', 'InkedGVS_212_W_T2_1_Trim', 'InkedGVS_213_W_T1_1_Trim', 'InkedGVS_213_W_T2_1_Trim', \
  'InkedGVS_214_W_T1_1_Trim', 'InkedGVS_214_W_T2_1_Trim', 'InkedGVS_215_W_T1_1_Trim', 'InkedGVS_215_W_T2_1_Trim', \
  'InkedGVS_216_W_T1_1_Trim', 'InkedGVS_216_W_T2_1_Trim', 'InkedGVS_218_W_T1_1_Trim', 'InkedGVS_218_W_T2_1_Trim', \
  'InkedGVS_219_W_T1_1_Trim', 'InkedGVS_219_W_T2_1_Trim', \
  'InkedGVS_310_T_T1_1_Trim', 'InkedGVS_310_T_T2_1_Trim', 'InkedGVS_311_T_T1_1_Trim', 'InkedGVS_311_T_T2_1_Trim', \
  'InkedGVS_312_T_T1_Trim', 'InkedGVS_312_T_T2_Trim', 'InkedGVS_314_T_T1_1_Trim', 'InkedGVS_314_T_T2_1_Trim', \
  'InkedGVS_318_T_T1_Trim', 'InkedGVS_318_T_T2_Trim', 'InkedGVS_320_T_T1_1_Trim', 'InkedGVS_320_T_T2_1_Trim', \
  'InkedGVS_321_T_T1_1_Trim', 'InkedGVS_321_T_T2_1_Trim', 'InkedGVS_323_T_T1_1_Trim', \
  'InkedGVS_310_W_T1_1_Trim', 'InkedGVS_310_W_T2_1_Trim', 'InkedGVS_311_W_T1_1_Trim',\
   'InkedGVS_311_W_T2_1', 'InkedGVS_312_W_T2_Trim', 'InkedGVS_313_W_T1_1_Trim', 'InkedGVS_313_W_T2_1_Trim', \
  'InkedGVS_314_W_T1_1_Trim', 'InkedGVS_314_W_T2_1_Trim', 'InkedGVS_318_W_T1_Trim', 'InkedGVS_318_W_T2_Trim', \
  'InkedGVS_320_W_T1_1_Trim', 'InkedGVS_320_W_T2_1_Trim', 'InkedGVS_321_W_T1_1_Trim', 'InkedGVS_321_W_T2_1_Trim', \
  'InkedGVS_322_W_T1_1_Trim', 'InkedGVS_322_W_T2_1_Trim', 'InkedGVS_323_W_T1_1_Trim',\
   'InkedGVS_323_W_T2_1_Trim', 'InkedGVS_403_T_T1_1_Trim', 'InkedGVS_403_T_T2_1_Trim', 'InkedGVS_406_T_T1_1_Trim', \
  'InkedGVS_406_T_T2_1_Trim', 'InkedGVS_407_T_T1_1_Trim', 'InkedGVS_407_T_T2_1_Trim', 'InkedGVS_408_T_T1_1_Trim', \
  'InkedGVS_408_T_T2_1_Trim', 'InkedGVS_403_W_T2_1_Trim', 'InkedGVS_405_W_T1_Trim', 'InkedGVS_405_W_T2_Trim', \
  'InkedGVS_405_W_T3_Trim', 'InkedGVS_405_W_T4_Trim', 'InkedGVS_406_W_T1_1_Trim', 'InkedGVS_406_W_T2_1_Trim', \
  'InkedGVS_407_W_T1_1_Trim', 'InkedGVS_407_W_T2_1_Trim', 'InkedGVS_408_W_T1_1_Trim', 'InkedGVS_408_W_T2_1_Trim'],\
                  array1), 
  **dict.fromkeys(['InkedGVS_217_T_T1_1_Trim', 'InkedGVS_217_T_T2_1_Trim', 'InkedGVS_409_T_T1_Trim', \
                  'InkedGVS_409_T_T2_Trim'], array2),
  **dict.fromkeys(['InkedGVS_217_W_T1_1_Trim', 'InkedGVS_217_W_T2_1_Trim', 'InkedGVS_410_T_T1_1_Trim', \
                  'InkedGVS_410_T_T2_1_Trim', 'InkedGVS_404_W_T1_1_Trim', 'InkedGVS_404_W_T2_1_Trim', \
                  'InkedGVS_404_W_T3_1_Trim', 'InkedGVS_404_W_T4_1_Trim', 'InkedGVS_409_W_T1_Trim', \
                  'InkedGVS_409_W_T2_Trim', 'InkedGVS_410_W_T1_1_Trim', 'InkedGVS_410_W_T2_1_Trim', \
                  'InkedGVS_411_W_T1_1_Trim', 'InkedGVS_411_W_T2_1_Trim', 'InkedGVS_411_W_T3_1_Trim', \
                  'InkedGVS_411_W_T4_1_Trim', 'InkedGVS_102_W_T1_1_Trim'], array3),
    **dict.fromkeys(['InkedGVS_112_W_T1_1_Trim', 'InkedGVS_113_W_T1_1_Trim', 'InkedGVS_115_W_T1_1_Trim', \
                    'InkedGVS_123_W_T1_1_Trim', 'InkedGVS_124_W_T1_1_Trim'], array4), 
}

#Make a dictionary for all videos, where the key is the video name and value is corresponding scales for 
#hip, knee, ankles, toes and heels 
scale_hip1 = [[(37+65+300)], [(37+65+500)], [(37+65+500)]]
scale_knee1 =  [[(37+65+300)], [(37+65+700)], [(37+65+520)]]
scale_feet1 = [[(37+65+300)], [(37+65+1000)], [(37+65+590)]]
scale_feet2 = [[(37+65+300)], [(37+65+300)], [(37+65+550)]]

scales_hip = {
  **dict.fromkeys(['InkedGVS_212_T_T1_1_Trim',  'InkedGVS_212_T_T2_1_Trim', 'InkedGVS_213_T_T1_1_Trim',\
 'InkedGVS_213_T_T2_1_Trim', 'InkedGVS_214_T_T1_1_Trim', 'InkedGVS_214_T_T2_1_Trim',\
    'InkedGVS_215_T_T1_1_Trim', 'InkedGVS_215_T_T2_1_Trim', 'InkedGVS_216_T_T1_1_Trim', 'InkedGVS_216_T_T2_1_Trim', \
      'InkedGVS_217_T_T1_1_Trim', 'InkedGVS_217_T_T2_1_Trim', 'InkedGVS_218_T_T1_1_Trim', \
      'InkedGVS_218_T_T2_1_Trim', 'InkedGVS_219_T_T1_1_Trim', 'InkedGVS_219_T_T2_1_Trim', 'InkedGVS_212_W_T1_1_Trim',\
      'InkedGVS_212_W_T2_1_Trim', 'InkedGVS_213_W_T1_1_Trim', 'InkedGVS_213_W_T2_1_Trim', \
      'InkedGVS_214_W_T1_1_Trim', 'InkedGVS_214_W_T2_1_Trim', 'InkedGVS_215_W_T1_1_Trim', \
      'InkedGVS_215_W_T2_1_Trim', 'InkedGVS_216_W_T1_1_Trim', 'InkedGVS_216_W_T2_1_Trim', \
       'InkedGVS_217_W_T1_1_Trim', 'InkedGVS_217_W_T2_1_Trim', 'InkedGVS_218_W_T1_1_Trim', \
       'InkedGVS_218_W_T2_1_Trim', 'InkedGVS_219_W_T1_1_Trim', 'InkedGVS_219_W_T2_1_Trim', \
      'InkedGVS_310_T_T1_1_Trim', 'InkedGVS_310_T_T2_1_Trim', 'InkedGVS_311_T_T1_1_Trim', \
      'InkedGVS_311_T_T2_1_Trim', 'InkedGVS_312_T_T1_Trim', 'InkedGVS_312_T_T2_Trim', \
      'InkedGVS_314_T_T1_1_Trim', 'InkedGVS_314_T_T2_1_Trim', 'InkedGVS_318_T_T1_Trim', \
      'InkedGVS_318_T_T2_Trim', 'InkedGVS_320_T_T1_1_Trim', 'InkedGVS_320_T_T2_1_Trim', \
      'InkedGVS_321_T_T1_1_Trim', 'InkedGVS_321_T_T2_1_Trim', 'InkedGVS_323_T_T1_1_Trim', \
      'InkedGVS_310_W_T1_1_Trim', 'InkedGVS_310_W_T2_1_Trim', 'InkedGVS_311_W_T1_1_Trim', \
      'InkedGVS_311_W_T2_1', 'InkedGVS_312_W_T2_Trim', 'InkedGVS_313_W_T1_1_Trim', \
      'InkedGVS_313_W_T2_1_Trim', 'InkedGVS_314_W_T1_1_Trim', 'InkedGVS_314_W_T2_1_Trim', \
      'InkedGVS_318_W_T1_Trim', 'InkedGVS_318_W_T2_Trim', 'InkedGVS_320_W_T1_1_Trim', \
      'InkedGVS_320_W_T2_1_Trim', 'InkedGVS_321_W_T1_1_Trim', 'InkedGVS_321_W_T2_1_Trim', \
      'InkedGVS_322_W_T1_1_Trim', 'InkedGVS_322_W_T2_1_Trim', 'InkedGVS_323_W_T1_1_Trim', \
      'InkedGVS_323_W_T2_1_Trim', 'InkedGVS_403_T_T1_1_Trim', 'InkedGVS_403_T_T2_1_Trim', \
       'InkedGVS_406_T_T1_1_Trim', 'InkedGVS_406_T_T2_1_Trim', 'InkedGVS_407_T_T1_1_Trim', \
      'InkedGVS_407_T_T2_1_Trim', 'InkedGVS_408_T_T1_1_Trim', 'InkedGVS_408_T_T2_1_Trim', \
      'InkedGVS_409_T_T1_Trim', 'InkedGVS_409_T_T2_Trim', 'InkedGVS_410_T_T1_1_Trim', \
      'InkedGVS_410_T_T2_1_Trim', 'InkedGVS_403_W_T2_1_Trim', 'InkedGVS_404_W_T1_1_Trim', \
      'InkedGVS_404_W_T2_1_Trim', 'InkedGVS_404_W_T3_1_Trim', 'InkedGVS_404_W_T4_1_Trim', \
      'InkedGVS_405_W_T1_Trim', 'InkedGVS_405_W_T2_Trim', 'InkedGVS_405_W_T3_Trim', \
      'InkedGVS_405_W_T4_Trim', 'InkedGVS_406_W_T1_1_Trim', 'InkedGVS_406_W_T2_1_Trim', 
      'InkedGVS_407_W_T1_1_Trim', 'InkedGVS_407_W_T2_1_Trim', 'InkedGVS_408_W_T1_1_Trim', \
      'InkedGVS_408_W_T2_1_Trim', 'InkedGVS_409_W_T1_Trim', 'InkedGVS_409_W_T2_Trim', \
      'InkedGVS_410_W_T1_1_Trim', 'InkedGVS_410_W_T2_1_Trim', 'InkedGVS_411_W_T1_1_Trim', \
      'InkedGVS_411_W_T2_1_Trim', 'InkedGVS_411_W_T3_1_Trim', 'InkedGVS_411_W_T4_1_Trim', \
      'InkedGVS_102_W_T1_1_Trim', 'InkedGVS_112_W_T1_1_Trim',  'InkedGVS_113_W_T1_1_Trim', \
      'InkedGVS_115_W_T1_1_Trim', 'InkedGVS_123_W_T1_1_Trim', 'InkedGVS_124_W_T1_1_Trim'], scale_hip1), 
  **dict.fromkeys([], )
}

scales_knee = {
  **dict.fromkeys(['InkedGVS_212_T_T1_1_Trim',  'InkedGVS_212_T_T2_1_Trim', 'InkedGVS_213_T_T1_1_Trim',\
 'InkedGVS_213_T_T2_1_Trim', 'InkedGVS_214_T_T1_1_Trim', 'InkedGVS_214_T_T2_1_Trim',\
    'InkedGVS_215_T_T1_1_Trim', 'InkedGVS_215_T_T2_1_Trim', 'InkedGVS_216_T_T1_1_Trim', 'InkedGVS_216_T_T2_1_Trim', \
      'InkedGVS_217_T_T1_1_Trim', 'InkedGVS_217_T_T2_1_Trim', 'InkedGVS_218_T_T1_1_Trim', \
      'InkedGVS_218_T_T2_1_Trim', 'InkedGVS_219_T_T1_1_Trim', 'InkedGVS_219_T_T2_1_Trim', \
      'InkedGVS_212_W_T1_1_Trim', 'InkedGVS_212_W_T2_1_Trim', 'InkedGVS_213_W_T1_1_Trim', 'InkedGVS_213_W_T2_1_Trim', \
      'InkedGVS_214_W_T1_1_Trim', 'InkedGVS_214_W_T2_1_Trim', 'InkedGVS_215_W_T1_1_Trim', \
      'InkedGVS_215_W_T2_1_Trim', 'InkedGVS_216_W_T1_1_Trim', 'InkedGVS_216_W_T2_1_Trim', \
      'InkedGVS_217_W_T1_1_Trim', 'InkedGVS_217_W_T2_1_Trim', 'InkedGVS_218_W_T1_1_Trim', \
      'InkedGVS_218_W_T2_1_Trim', 'InkedGVS_219_W_T1_1_Trim', 'InkedGVS_219_W_T2_1_Trim', \
      'InkedGVS_310_T_T1_1_Trim', 'InkedGVS_310_T_T2_1_Trim', 'InkedGVS_311_T_T1_1_Trim', \
      'InkedGVS_311_T_T2_1_Trim', 'InkedGVS_312_T_T1_Trim', 'InkedGVS_312_T_T2_Trim', \
      'InkedGVS_314_T_T1_1_Trim', 'InkedGVS_314_T_T2_1_Trim', 'InkedGVS_318_T_T1_Trim', \
       'InkedGVS_318_T_T2_Trim', 'InkedGVS_320_T_T1_1_Trim', 'InkedGVS_320_T_T2_1_Trim', \
      'InkedGVS_321_T_T1_1_Trim', 'InkedGVS_321_T_T2_1_Trim', 'InkedGVS_323_T_T1_1_Trim', \
      'InkedGVS_310_W_T1_1_Trim', 'InkedGVS_310_W_T2_1_Trim', 'InkedGVS_311_W_T1_1_Trim', \
      'InkedGVS_311_W_T2_1', 'InkedGVS_312_W_T2_Trim', 'InkedGVS_313_W_T1_1_Trim', \
      'InkedGVS_313_W_T2_1_Trim', 'InkedGVS_314_W_T1_1_Trim', 'InkedGVS_314_W_T2_1_Trim', \
      'InkedGVS_318_W_T1_Trim', 'InkedGVS_318_W_T2_Trim', 'InkedGVS_320_W_T1_1_Trim', \
      'InkedGVS_320_W_T2_1_Trim', 'InkedGVS_321_W_T1_1_Trim', 'InkedGVS_321_W_T2_1_Trim', \
      'InkedGVS_322_W_T1_1_Trim', 'InkedGVS_322_W_T2_1_Trim', 'InkedGVS_323_W_T1_1_Trim', \
      'InkedGVS_323_W_T2_1_Trim', 'InkedGVS_403_T_T1_1_Trim', 'InkedGVS_403_T_T2_1_Trim', \
      'InkedGVS_406_T_T1_1_Trim', 'InkedGVS_406_T_T2_1_Trim', 'InkedGVS_407_T_T1_1_Trim', 
      'InkedGVS_407_T_T2_1_Trim', 'InkedGVS_408_T_T1_1_Trim', 'InkedGVS_408_T_T2_1_Trim', \
      'InkedGVS_409_T_T1_Trim', 'InkedGVS_409_T_T2_Trim', 'InkedGVS_410_T_T1_1_Trim', \
      'InkedGVS_410_T_T2_1_Trim', 'InkedGVS_403_W_T2_1_Trim', 'InkedGVS_404_W_T1_1_Trim', \
      'InkedGVS_404_W_T2_1_Trim', 'InkedGVS_404_W_T3_1_Trim', 'InkedGVS_404_W_T4_1_Trim', 
      'InkedGVS_405_W_T1_Trim', 'InkedGVS_405_W_T2_Trim', 'InkedGVS_405_W_T3_Trim', \
      'InkedGVS_405_W_T4_Trim', 'InkedGVS_406_W_T1_1_Trim', 'InkedGVS_406_W_T2_1_Trim', 
      'InkedGVS_407_W_T1_1_Trim', 'InkedGVS_407_W_T2_1_Trim', 'InkedGVS_408_W_T1_1_Trim', \
      'InkedGVS_408_W_T2_1_Trim', 'InkedGVS_409_W_T1_Trim', 'InkedGVS_409_W_T2_Trim', \
      'InkedGVS_410_W_T1_1_Trim', 'InkedGVS_410_W_T2_1_Trim', 'InkedGVS_411_W_T1_1_Trim', \
      'InkedGVS_411_W_T2_1_Trim', 'InkedGVS_411_W_T3_1_Trim', 'InkedGVS_411_W_T4_1_Trim', \
      'InkedGVS_102_W_T1_1_Trim', 'InkedGVS_112_W_T1_1_Trim',  'InkedGVS_113_W_T1_1_Trim', \
      'InkedGVS_115_W_T1_1_Trim', 'InkedGVS_123_W_T1_1_Trim', 'InkedGVS_124_W_T1_1_Trim'], scale_knee1), 
  **dict.fromkeys([], )
}

scales_feet = {
  **dict.fromkeys(['InkedGVS_212_T_T1_1_Trim',  'InkedGVS_212_T_T2_1_Trim', 'InkedGVS_213_T_T1_1_Trim',\
 'InkedGVS_213_T_T2_1_Trim', 'InkedGVS_214_T_T1_1_Trim', 'InkedGVS_214_T_T2_1_Trim',\
    'InkedGVS_215_T_T1_1_Trim', 'InkedGVS_215_T_T2_1_Trim', 'InkedGVS_216_T_T1_1_Trim', 'InkedGVS_216_T_T2_1_Trim', \
  'InkedGVS_218_T_T1_1_Trim', 'InkedGVS_218_T_T2_1_Trim', 'InkedGVS_219_T_T1_1_Trim', 'InkedGVS_219_T_T2_1_Trim', \
  'InkedGVS_212_W_T1_1_Trim', 'InkedGVS_212_W_T2_1_Trim', 'InkedGVS_213_W_T1_1_Trim', 'InkedGVS_213_W_T2_1_Trim', \
  'InkedGVS_214_W_T1_1_Trim', 'InkedGVS_214_W_T2_1_Trim', 'InkedGVS_215_W_T1_1_Trim', \
  'InkedGVS_215_W_T2_1_Trim', 'InkedGVS_216_W_T1_1_Trim', 'InkedGVS_216_W_T2_1_Trim', \
  'InkedGVS_218_W_T1_1_Trim', 'InkedGVS_218_W_T2_1_Trim', 'InkedGVS_219_W_T1_1_Trim', 'InkedGVS_219_W_T2_1_Trim', \
  'InkedGVS_310_T_T1_1_Trim', 'InkedGVS_310_T_T2_1_Trim', 'InkedGVS_311_T_T1_1_Trim', \
  'InkedGVS_311_T_T2_1_Trim', 'InkedGVS_312_T_T1_Trim', 'InkedGVS_312_T_T2_Trim', \
  'InkedGVS_314_T_T1_1_Trim', 'InkedGVS_314_T_T2_1_Trim', 'InkedGVS_318_T_T1_Trim', \
  'InkedGVS_318_T_T2_Trim', 'InkedGVS_320_T_T1_1_Trim', 'InkedGVS_320_T_T2_1_Trim', \
  'InkedGVS_321_T_T1_1_Trim', 'InkedGVS_321_T_T2_1_Trim', 'InkedGVS_323_T_T1_1_Trim', \
  'InkedGVS_310_W_T1_1_Trim', 'InkedGVS_310_W_T2_1_Trim', 'InkedGVS_311_W_T1_1_Trim', \
  'InkedGVS_311_W_T2_1', 'InkedGVS_312_W_T2_Trim', 'InkedGVS_313_W_T1_1_Trim', \
  'InkedGVS_313_W_T2_1_Trim', 'InkedGVS_314_W_T1_1_Trim', 'InkedGVS_314_W_T2_1_Trim', \
  'InkedGVS_318_W_T1_Trim', 'InkedGVS_318_W_T2_Trim', 'InkedGVS_320_W_T1_1_Trim', \
  'InkedGVS_320_W_T2_1_Trim', 'InkedGVS_321_W_T1_1_Trim', 'InkedGVS_321_W_T2_1_Trim', \
  'InkedGVS_322_W_T1_1_Trim', 'InkedGVS_322_W_T2_1_Trim', 'InkedGVS_323_W_T1_1_Trim', \
  'InkedGVS_323_W_T2_1_Trim', 'InkedGVS_403_T_T1_1_Trim', 'InkedGVS_403_T_T2_1_Trim', \
  'InkedGVS_406_T_T1_1_Trim', 'InkedGVS_406_T_T2_1_Trim', 'InkedGVS_407_T_T1_1_Trim', 
  'InkedGVS_407_T_T2_1_Trim', 'InkedGVS_408_T_T1_1_Trim', 'InkedGVS_408_T_T2_1_Trim', \
  'InkedGVS_403_W_T2_1_Trim', 'InkedGVS_405_W_T1_Trim', 'InkedGVS_405_W_T2_Trim', \
  'InkedGVS_405_W_T3_Trim', 'InkedGVS_405_W_T4_Trim', 'InkedGVS_406_W_T1_1_Trim', \
  'InkedGVS_406_W_T2_1_Trim', 'InkedGVS_407_W_T1_1_Trim', 'InkedGVS_407_W_T2_1_Trim', \
  'InkedGVS_408_W_T1_1_Trim', 'InkedGVS_408_W_T2_1_Trim'],\
                  scale_feet1), 
  **dict.fromkeys(['InkedGVS_217_T_T1_1_Trim', 'InkedGVS_217_T_T2_1_Trim', 'InkedGVS_217_W_T1_1_Trim', \
                   'InkedGVS_217_W_T2_1_Trim', 'InkedGVS_409_T_T1_Trim', 'InkedGVS_409_T_T2_Trim', \
                  'InkedGVS_410_T_T1_1_Trim', 'InkedGVS_410_T_T2_1_Trim', 'InkedGVS_404_W_T1_1_Trim', \
                  'InkedGVS_404_W_T2_1_Trim', 'InkedGVS_404_W_T3_1_Trim', 'InkedGVS_404_W_T4_1_Trim', \
                  'InkedGVS_409_W_T1_Trim', 'InkedGVS_409_W_T2_Trim', 'InkedGVS_410_W_T1_1_Trim', \
                  'InkedGVS_410_W_T2_1_Trim', 'InkedGVS_411_W_T1_1_Trim', 'InkedGVS_411_W_T2_1_Trim', \
                  'InkedGVS_411_W_T3_1_Trim', 'InkedGVS_411_W_T4_1_Trim', 'InkedGVS_102_W_T1_1_Trim', \
                  'InkedGVS_112_W_T1_1_Trim',  'InkedGVS_113_W_T1_1_Trim', 'InkedGVS_115_W_T1_1_Trim', \
                  'InkedGVS_123_W_T1_1_Trim', 'InkedGVS_124_W_T1_1_Trim'], scale_feet2)
}

In [49]:
for cohort in cohorts:
    for trial in trials:
        for camera in cameras[1:]: #Front look camera only
            frame_path = raw_path+cohort+trial+camera #Path to save the frames to 
            videos = os.listdir(frame_path)
#             print (len(videos))
            for video in videos:
                print (video)
                frames = glob.glob(frame_path+video+'\\processed2d\\*.csv')
                start_time = time.time()
                for frame in frames:
#                     print (frame)
                    data_frame = pd.DataFrame()
#                     im_path = frame_path+video+'\\'+frame
                    try:
                        frame_csv = pd.read_csv(frame, index_col = 0)
#                         print (frame_csv)
                        coords = frame_csv[['x', 'y']].values
                        #Only reading the x, y coordinates for 12 locations 
                        array = arrays[video]
                        data_frame=pd.DataFrame(columns = ['x', 'y', 'z'])
                        for idx, coord in enumerate(coords):
                            if idx in [0, 3]: #Hip 
                                scale = scales_hip[video]
                            elif idx in [1, 4]: #Knee
                                scale = scales_knee[video]
                            else: #Ankle, Toes and Heel
                                scale = scales_feet[video]
                            data_frame.loc[len(data_frame)] = np.array(convert2d_to_3d(coord, scale, array))[0].T
                        data_frame.index = order
                        data_frame['confidence'] = frame_csv['confidence']
#                         print (data_frame)
                        data_frame.to_csv(frame_path3d+cohort+trial+camera+video+'\\'+frame.split('\\')[-1][:-3]+'csv')
                    except Exception as e:
                        print (e)
                print ('Time taken to process 1 video = ', time.time()- start_time)

InkedGVS_124_W_T1_1_Trim
Time taken to process 1 video =  208.65262842178345


### Side look camera (_0)

In [ ]:
#Take care while 3D converting 0's in 2D i.e. non-existing points 
